In [18]:
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split

data = pd.read_csv('./data_v3.csv')

data['bug_num'] = data['bug_num'].map(lambda bnum: bnum - 1)

In [19]:
train, test = train_test_split(data)

In [22]:
'''
created_at,
updated_at,
size,
stargazers_count,
watchers_count, #
forks_count, #
network_count, #
subscribers_count, #
followers, #
public_repos, #
created_at,
contributions, #
author_id,
LOC, #
Add, #
Delete, #
Files, #
bug_num  #
'''

feature_columns = [
    #'watchers_count',
    #'forks_count',
    #'network_count',
    #'subscribers_count',
    'followers',
    'public_repos',
    'contributions',
    'LOC',
    'Add',
    'Delete',
    'Files',
]

target_column = 'bug_num'


xgtrain = xgb.DMatrix(train[feature_columns].values, train[target_column].values)
xgtest = xgb.DMatrix(test[feature_columns].values, test[target_column].values)


param = {
    'max_depth':5,
    'eta':0.1,
    'silent':1,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'objective':'multi:softmax',
    'num_class': 4, 
}

# build the model
watchlist  = [(xgtest,'eval'), (xgtrain,'train')]
num_round = 30
bst = xgb.train(param, xgtrain, num_round, watchlist)


[18:14:33] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:14:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	eval-mlogloss:1.30922	train-mlogloss:1.30438
[1]	eval-mlogloss:1.24812	train-mlogloss:1.21831
[2]	eval-mlogloss:1.19471	train-mlogloss:1.13699
[3]	eval-mlogloss:1.15006	train-mlogloss:1.07048
[4]	eval-mlogloss:1.11473	train-mlogloss:1.00944
[5]	eval-mlogloss:1.07280	train-mlogloss:0.95941
[6]	eval-mlogloss:1.03895	train-mlogloss:0.90856
[7]	eval-mlogloss:1.01413	train-mlogloss:0.86908
[8]	eval-mlogloss:0.98495	trai

In [23]:
# prediction and check the accuracy
preds = bst.predict(xgtest)
labels = xgtest.get_label()
print ('error%f' % \
       (sum(1 for i in range(len(preds)) if int(preds[i]>0.5)!=labels[i]) /float(len(preds))))


error0.400000
